## Metaclasses

Metaclass, the class of a class ([docs](https://docs.python.org/3/reference/datamodel.html#metaclasses))

Basic class creation:

> By default, classes are constructed using type(). The class body is executed in a new namespace and the class name is bound locally to the result of type(name, bases, namespace).

When a class definition is executed, the following steps occur:

* MRO entries are resolved;
* the appropriate metaclass is determined;
* the class namespace is prepared;
* the class body is executed;
* the class object is created.

Also:

“Metaclasses are deeper magic than 99% of users should ever worry about. If you wonder whether you need them, you don’t (the people who actually need them know with certainty that they need them, and don’t need an explanation about why).”

In [2]:
class Meta(type):
    pass

class MyClass(metaclass=Meta):
    pass

class MySubclass(MyClass):
    pass

In [7]:
type(MySubclass) # __main__.Meta

__main__.Meta

In [10]:
a = MyClass()

> type is a metaclass, of which classes are instances

We can create a class with type, like so:


In [14]:
C = type("C", (), {})

In [15]:
type(C)

type

What happens at class creation?

In [16]:
class A:
    pass

a = A()

* A is called, it execute `__call__` of superclass, which is type
* parent's `__call__` invokes `__new__` and `__init__`
* a will be set to what `__new__` returns

So `type` is a class as well, so it has a `__new__` method.

In [18]:
type.__new__?

In [42]:
def new(cls):
    inst = object.__new__(cls)
    inst.attr = 1
    return inst
    

In [43]:
class A:
    pass


In [44]:
A.__new__ = new

In [45]:
a = A()

In [33]:
a.attr

1

However, we cannot alter `type` here.


In [46]:
type.__new__

<function type.__new__(*args, **kwargs)>

In [48]:
# type.__new__ = lambda cls: cls # TypeError: cannot set '__new__' attribute of immutable type 'type'

In order to customize class creation, we can use a metaclass.

In [53]:
class Meta(type):
    def __new__(cls, name, bases, dct):
        v = super().__new__(cls, name, bases, dct)
        v.attr = 123
        return v

In [54]:
class A(metaclass=Meta):
    pass

In [55]:
type(A)

__main__.Meta

In [56]:
A.attr

123

In [57]:
A.__bases__

(object,)

A more complete example

In [78]:
class MyMeta(type):
    def __new__(cls, name, bases, dct):
        print(f"__new__ metaclass {cls} {name} {bases} {dct}")
        inst = super().__new__(cls, name, bases, dct)
        return inst

class A(metaclass=MyMeta):
    def __new__(cls, *args, **kwargs):
        print(f"__new__ {cls} {args} {kwargs}")
        return super().__new__(cls, *args, **kwargs)
    
    def __init__(self):
        print(f"__init__ {self}")
    

__new__ metaclass <class '__main__.MyMeta'> A () {'__module__': '__main__', '__qualname__': 'A', '__new__': <function A.__new__ at 0x7fd9e9085620>, '__init__': <function A.__init__ at 0x7fd9e9087ec0>, '__classcell__': <cell at 0x7fd9f8290850: empty>}


In [79]:
a = A()

__new__ <class '__main__.A'> () {}
__init__ <__main__.A object at 0x7fd9e9655cd0>


In [77]:
a.x = 1

Example: A metaclass that disallows undocumented classes.

In [66]:
class DocsRequired(type):
    def __new__(cls, name, bases, dct):
        inst = super().__new__(cls, name, bases, dct)
        if not inst.__doc__:
            raise RuntimeError(f"class has no docs {cls}")
        return inst

In [80]:
# class A(metaclass=DocsRequired): # RuntimeError: class has no docs <class '__main__.DocsRequired'>
#     pass

Some use cases for registration.

* registration, example: [task.py](https://github.com/spotify/luigi/blob/38b0c2b8d400f6fc57ebbdc4064aadd4f3d7b612/luigi/task.py#L147-L171), [task_register.py](https://github.com/spotify/luigi/blob/38b0c2b8d400f6fc57ebbdc4064aadd4f3d7b612/luigi/task_register.py#L39-L48)
* enforcing implementations of abstract methods, example [metrics.py](https://github.com/spotify/luigi/blob/38b0c2b8d400f6fc57ebbdc4064aadd4f3d7b612/luigi/metrics.py#L44-L73)

Abstract base classes, ABC provides a metaclass as well.

Module [abc]() provides a convenience layer as `abc.ABC` -- [abc.py](https://github.com/python/cpython/blob/91a8e002c21a5388c5152c5a4871b9a2d85f0fc1/Lib/abc.py#L184-L188)

Used in conjuntion with `abc.abstractmethod` class decorator to ensure all required methods are implemented.

> Requires that the metaclass is ABCMeta or derived from it.  A
    class that has a metaclass derived from ABCMeta cannot be
    instantiated unless all of its abstract methods are overridden.